In [ ]:
using DrWatson
@quickactivate("HyperspectraWithNeXL")

using NeXLSpectrum
using Gadfly
using DataFrames

In [ ]:
path = joinpath(datadir(),"exp_raw","K2496")
# Read the spectra from disk
k2496 = loadspectrum.(joinpath(path,"K2496_$(i).msa") for i in 1:3)

In [ ]:
k2496_mat = parse(Material,"0.323*O+0.2291*Si+0.018*Ti+0.4299*Ba",name="K2496 nominal")

In [ ]:
e0 = 10.0e3
eff = SDDEfficiency(AP33Tabulation(); thickness=0.0370, deadlayer=30.0e-7, entrance=Film(pure(n"Al"), 10.0e-7))

In [ ]:
cmod = ContinuumModel(k2496_mat, e0, deg2rad(40.0))
  set_default_plot_size(8inch,3inch)
plot([ea->emitted(cmod, ea),ea->generated(cmod, ea)],100.0,e0)

In [ ]:
det = BasicEDS(4096, -480.40409, 5.00525, 132.0, 110, 
         Dict(KShell=>n"B", LShell=>n"Ca", MShell=>n"Cs"))
resp = NeXLSpectrum.detectorresponse(det, eff)
emt = map(ch->ch>=lld(det) ? emitted(cmod,energy(ch,det)) : 0.0,1:channelcount(det))
meas =  resp*emt
lyrs = [ layer(x=energyscale(det), y=emt, Geom.line, Theme(default_color="red")), layer(x=energyscale(det), y=meas, Geom.line) ]
plot(lyrs..., Coord.cartesian(xmin=0, xmax=e0))

In [ ]:
roi = channel(2.0e3,det):channel(3.7e3,det)
model, mc = Castellano2004a, Riveros1993
foreach(s->s[:Composition]=k2496_mat, k2496)
brem = [ fitcontinuum(spec, resp, [roi], brem=model, mc=mc) for spec in k2496 ]

In [ ]:
plot(k2496..., brem..., klms = [ n"C", n"O", n"Si", n"Ba" ], yscale=1.02, xmax=2.5e3)

In [ ]:
iroi = 0.45e3:0.55e3
sum(k2496[1][iroi])-sum(brem[1][iroi])

In [ ]:
benitoite = loadspectrum(joinpath(path,"Benitoite std.msa"))
benitoite[:Composition] = mat"BaTiSi3O9"
brem2 = fitcontinuum(benitoite, resp, [roi], brem=model, mc=mc)
plot(benitoite, brem2, klms = [ n"C", n"O", n"Si", n"Ba" ], yscale=1.05, xmax=2.5e3)


In [ ]:
sum(benitoite[iroi])-sum(brem2[iroi])

In [ ]:
iroi = 0.45e3:0.55e3
k_o = [ (sum(k2496[i][iroi])-sum(brem[i][iroi]))/(sum(benitoite[iroi])-sum(brem2[iroi])) for i in 1:3 ]

In [ ]:
using Statistics
mean(k_o)

In [ ]:
iroi = 1.7e3:1.8e3
k_si = [ (sum(k2496[i][iroi])-sum(brem[i][iroi]))/(sum(benitoite[iroi])-sum(brem2[iroi])) for i in 1:3 ]

In [ ]:
using Statistics
mean(k_si)